In [13]:
import pandas as pd
from geopy.distance import great_circle
from shapely.geometry import Point, LineString
from shapely.ops import transform
import geopandas as gpd
import pandas as pd
import pyproj

TARGET_DISTANCE = 50 # In nautical miles
input_list = "YSSY YBBN" # ICAO codes of the start and end airports

# Read the CSV file into a DataFrame
df = pd.read_csv(r"C:\Users\arsla\Desktop\MY LIFE\pf-notams\raw\airport_locations.csv")

# Split the input list into individual airport codes
airport_codes = input_list.split()

# Get the coordinates of the start and end airports
start_airport = df[df['ICAO'] == airport_codes[0]]
end_airport = df[df['ICAO'] == airport_codes[1]]

# Create a LineString representing the great circle path between the two airports
line = LineString([(start_airport['Longitude'], start_airport['Latitude']), 
                    (end_airport['Longitude'], end_airport['Latitude'])])

# Create a buffer around the line
wgs84 = pyproj.CRS('EPSG:4326')
aeqd = pyproj.CRS(f"+proj=aeqd +lat_0={line.centroid.y} +lon_0={line.centroid.x}")
project = pyproj.Transformer.from_crs(wgs84, aeqd, always_xy=True).transform
buffer = transform(project, line).buffer(TARGET_DISTANCE / 60)  # Convert nautical miles to degrees

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']))

# Find the airports that fall within the buffer
airports_in_buffer = gdf[gdf.within(buffer)]

# Return the ICAO codes of the airports in the buffer as a string
print(' '.join(airports_in_buffer['ICAO']))



c:\Users\arsla\.conda\envs\azure\lib\site-packages\shapely\geometry\linestring.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(c) for c in o]


In [38]:
import pandas as pd
from geopy.distance import geodesic

TARGET_DISTANCE = 300000 # In nautical miles

# Load airports data from CSV file
df = pd.read_csv(r"C:\Users\arsla\Desktop\MY LIFE\pf-notams\raw\airport_locations.csv")

# Function to calculate distance between two coordinates
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).nautical

# Function to find the nearest airport given a set of coordinates
def find_nearest_airport(coord):
    distances = df[['Latitude', 'Longitude']].apply(lambda x: calculate_distance(coord, (x['Latitude'], x['Longitude'])), axis=1)
    return df.loc[distances.idxmin()]['ICAO']

def generate_extended_route(route):
    extended_route = []
    for i in range(len(route)-1):
        start_airport = route[i]
        end_airport = route[i+1]
        
        # Check if the airports are in the DataFrame
        if start_airport not in df['ICAO'].values or end_airport not in df['ICAO'].values:
            raise ValueError(f"Airport {start_airport} or {end_airport} not found in the CSV file.")
        
        start_coord = (df.loc[df['ICAO'] == start_airport]['Latitude'].values[0], df.loc[df['ICAO'] == start_airport]['Longitude'].values[0])
        end_coord = (df.loc[df['ICAO'] == end_airport]['Latitude'].values[0], df.loc[df['ICAO'] == end_airport]['Longitude'].values[0])
        
        extended_route.append(start_airport)
        
        # Calculate midpoint between the start and end airports
        midpoint = ((start_coord[0] + end_coord[0]) / 2, (start_coord[1] + end_coord[1]) / 2)
        
        # Find the nearest airport to the midpoint
        nearest_airport = find_nearest_airport(midpoint)
        
        # Add the nearest airport to the extended route
        extended_route.append(nearest_airport)
        
        # Calculate the buffer around the flight path
        line = LineString([start_coord, end_coord])
        buffer = line.parallel_offset(TARGET_DISTANCE / 60, 'left').union(line.parallel_offset(TARGET_DISTANCE / 60, 'right'))
        
        # Find the airports that fall within the buffer
        airports_in_buffer = gdf[gdf.within(buffer)]
        
        # Add the ICAO codes of the airports in the buffer to the extended route
        extended_route.extend(airports_in_buffer['ICAO'].tolist())
    
    extended_route.append(end_airport)
    return extended_route


In [39]:
# Example usage
input_route = "YBBN YSSY"
airports_route = input_route.split()

try:
    extended_route = generate_extended_route(airports_route)
    print("Extended Route:", " ".join(extended_route))
except ValueError as e:
    print(e)

Extended Route: YBBN YSCH YSSY


In [33]:
import pandas as pd
from geopy.distance import geodesic

# Load airports data from CSV file
df = pd.read_csv(r"C:\Users\arsla\Desktop\MY LIFE\pf-notams\raw\airport_locations.csv")

# Function to calculate distance between two coordinates
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).nautical

# Function to find the nearest airport given a set of coordinates
def find_nearest_airport(coord):
    distances = df[['Latitude', 'Longitude']].apply(lambda x: calculate_distance(coord, (x['Latitude'], x['Longitude'])), axis=1)
    return df.loc[distances.idxmin()]['ICAO']

# Function to generate the extended route
def generate_extended_route(route):
    extended_route = []
    for i in range(len(route)-1):
        start_airport = route[i]
        end_airport = route[i+1]
        
        # Check if the airports are in the DataFrame
        if start_airport not in df['ICAO'].values or end_airport not in df['ICAO'].values:
            raise ValueError(f"Airport {start_airport} or {end_airport} not found in the CSV file.")
        
        start_coord = (df.loc[df['ICAO'] == start_airport]['Latitude'].values[0], df.loc[df['ICAO'] == start_airport]['Longitude'].values[0])
        end_coord = (df.loc[df['ICAO'] == end_airport]['Latitude'].values[0], df.loc[df['ICAO'] == end_airport]['Longitude'].values[0])
        
        extended_route.append(start_airport)
        
        # Calculate midpoint between the start and end airports
        midpoint = ((start_coord[0] + end_coord[0]) / 2, (start_coord[1] + end_coord[1]) / 2)
        
        # Find the nearest airport to the midpoint
        nearest_airport = find_nearest_airport(midpoint)
        
        # Add the nearest airport to the extended route
        extended_route.append(nearest_airport)
    
    extended_route.append(end_airport)
    return extended_route

# Example usage
input_route = "YBBN YSSY"
airports_route = input_route.split()

try:
    extended_route = generate_extended_route(airports_route)
    print("Extended Route:", " ".join(extended_route))
except ValueError as e:
    print(e)


Extended Route: YBBN YSCH YSSY


In [52]:
import pandas as pd
from geopy.distance import great_circle

def find_airports_between(input_airports, offset_distance, airports_df):
    def calculate_distance(point1, point2):
        return great_circle(point1, point2).nautical

    def airports_in_zone(line_start, line_end, offset, airports):
        line_distance = great_circle(line_start, line_end).nautical
        line_direction = (line_end[0] - line_start[0], line_end[1] - line_start[1])
        unit_vector = (line_direction[1], -line_direction[0])
        unit_vector = (unit_vector[0] / line_distance, unit_vector[1] / line_distance)
        offset_vector = (unit_vector[0] * offset, unit_vector[1] * offset)
        min_lat = min(line_start[0], line_end[0]) + offset_vector[0]
        max_lat = max(line_start[0], line_end[0]) + offset_vector[0]
        min_lon = min(line_start[1], line_end[1]) + offset_vector[1]
        max_lon = max(line_start[1], line_end[1]) + offset_vector[1]

        filtered_airports = airports[
            (airports['Latitude'] >= min_lat) & (airports['Latitude'] <= max_lat) &
            (airports['Longitude'] >= min_lon) & (airports['Longitude'] <= max_lon)
        ]

        return filtered_airports

    airports = input_airports.split()
    departure_airport, arrival_airport = airports[0], airports[-1]

    departure_data = airports_df[airports_df['ICAO'] == departure_airport][['Latitude', 'Longitude']]
    arrival_data = airports_df[airports_df['ICAO'] == arrival_airport][['Latitude', 'Longitude']]

    if departure_data.empty or arrival_data.empty:
        return "Error: One or more airports not found in the CSV file."

    departure_coords = tuple(departure_data.values[0])
    arrival_coords = tuple(arrival_data.values[0])

    airports_in_zone_df = airports_in_zone(departure_coords, arrival_coords, offset_distance, airports_df)

    airport_list = [departure_airport] + list(airports_in_zone_df['ICAO']) + [arrival_airport]

    return " ".join(airport_list)

# Example usage
input_airports = "YBBN YSSY"
offset_distance = 0
result = find_airports_between(input_airports, offset_distance, pd.read_csv(r"C:\Users\arsla\Desktop\MY LIFE\pf-notams\raw\airport_locations.csv"))
print(result)


YBBN YBAF YBBN YBOK YSCH YSSY YSSY


In [69]:
def my_python_tool(input_airports: str) -> str:
    """
    Provided a string of airport codes as a list (eg. "YBBN YSSY"), return a list of airport codes that lie on the path between the two airports
    according to the Greater Circle Distance formula. 
     - The list should include the start and end airports. 
     - The list should be in order of the path.
    """
    offset_distance = 50  # nautical miles  
    airports_df = pd.read_csv(r"C:\Users\arsla\Desktop\MY LIFE\pf-notams\raw\airport_locations.csv")

    def calculate_distance(point1, point2):
        return great_circle(point1, point2).nautical

    def airports_in_zone(line_start, line_end, offset, airports):
        line_distance = great_circle(line_start, line_end).nautical
        line_direction = (line_end[0] - line_start[0], line_end[1] - line_start[1])
        unit_vector = (line_direction[1], -line_direction[0])
        unit_vector = (unit_vector[0] / line_distance, unit_vector[1] / line_distance)
        offset_vector = (unit_vector[0] * offset, unit_vector[1] * offset)
        min_lat = min(line_start[0], line_end[0]) + offset_vector[0]
        max_lat = max(line_start[0], line_end[0]) + offset_vector[0]
        min_lon = min(line_start[1], line_end[1]) + offset_vector[1]
        max_lon = max(line_start[1], line_end[1]) + offset_vector[1]

        filtered_airports = airports[
            (airports['Latitude'] >= min_lat) & (airports['Latitude'] <= max_lat) &
            (airports['Longitude'] >= min_lon) & (airports['Longitude'] <= max_lon)
        ]

        return filtered_airports
    airports_input = input_airports.replace('\n', '').strip()
    airports = input_airports.split()
    departure_airport, arrival_airport = airports[0], airports[-1]

    departure_data = airports_df[airports_df['ICAO'] == departure_airport][['Latitude', 'Longitude']]
    arrival_data = airports_df[airports_df['ICAO'] == arrival_airport][['Latitude', 'Longitude']]

    if departure_data.empty or arrival_data.empty:
        return "Error: One or more airports not found in the CSV file."

    departure_coords = tuple(departure_data.values[0])
    arrival_coords = tuple(arrival_data.values[0])

    airports_in_zone_df = airports_in_zone(departure_coords, arrival_coords, offset_distance, airports_df)

    airport_list = [departure_airport] + list(airports_in_zone_df['ICAO']) + [arrival_airport]

    # Remove duplicates
    unique_airports = []
    for airport in airport_list:
        if airport not in unique_airports:
            unique_airports.append(airport)

    return " ".join(unique_airports)

In [71]:
my_python_tool("\n\nYBBN YSSY")

'YBBN YBCG YSCH YSSY'